In [1]:
import numpy as np
import matplotlib.pyplot as plt, mpld3
from matplotlib import style
import pandas as pd 
import math

In [2]:
df=pd.read_csv('wdbc_data.csv',header=None)
X=df.loc[:,2:32].values

In [3]:
for feature_name in df.loc[:,2:32].columns:
    max_value = df[feature_name].max()
    min_value = df[feature_name].min()
    df[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)

In [4]:
X=df.loc[:,2:32].values
cat=df[1]
X.shape

(569, 30)

In [5]:
def kmeans(k, dm, X):
    
    print(f"purity, average distance, max distance to centroids and max distance between 2 feature vectors with L{dm} and k = {k}")
    centroids = {}
    for i in range(k):
        centroids[i] = X[i]
    while True:
        classes = {}
        prediction = []
        for j in range(k):
            classes[j] = []
        for row in range(0, len(X)):
            min_dist = math.inf
            c_index = math.inf
            for centroid in range(len(centroids)):
                sum = 0
                for col in range(0, 30):
                    diff = abs(X[row][col] -  centroids[centroid][col])
                    sum = sum + diff ** dm
                dist = sum ** (1/dm)
                if dist < min_dist:
                    min_dist = dist
                    c_index = centroid
            classes[c_index].append(X[row])
            prediction.append(c_index)
            
            
        previous = dict(centroids)
        
        for classification in classes:
            centroids[classification] = np.mean(classes[classification], axis = 0)
            
        isOptimal = True
        
        for centroid in range(0, len(centroids)):
            original_centroid = previous[centroid]
            curr = centroids[centroid]

            if np.array_equal(original_centroid, curr) == False:
                isOptimal = False
                
        if isOptimal:
            classes_global = classes
            break
    purity=[]    
    for centroid in range(k):
       
        m = 0
        b = 0
        for index in range(0, len(prediction)):
            if prediction[index] == centroid and cat[index] == 'B':
                b = b + 1
            if prediction[index] == centroid and cat[index] == 'M':
                m = m + 1
                
        pure= max(m,b)/ (m+b)
        purity.append(pure)
         
    dist_avg=[]
    distances=[]
    dist_max=[]
    for k in range(0,k):
        avg_dist=0
        max_dist=0
        for index in range(0, len(prediction)):
            if prediction[index] == k:
                sum = 0
                for col in range(0, 30):
                    diff = abs(X[index][col] -  centroids[k][col])
                    sum = sum + diff ** dm
                dist2 = sum ** (1/dm)
                distances.append(dist2)
        dist_avg.append(np.average(distances))
        dist_max.append(np.max(distances))
    

    max_dis = []
    for a in range(0,k):
        max_dist=0
        for index in range(len(prediction)):
            for ind in range(len(prediction)):
                if prediction[index] == a and prediction[ind] == a:
                    sum = 0
                    for col in range(0, 30):
                        diff = abs(X[index][col] -  X[ind][col])
                        sum = sum + diff ** dm
                        dist3 = sum ** (1/dm)
                        if dist3 > max_dist:
                            max_dist = dist3
        max_dis.append(max_dist)
    return purity,dist_avg,dist_max,max_dis  

In [6]:
kmeans(2,1,X)

purity, average distance, max distance to centroids and max distance between 2 feature vectors with L1 and k = 2


([0.9827586206896551, 0.8962025316455696],
 [3.068791559290859, 2.4211762105376637],
 [8.698698666848118, 8.698698666848118],
 [14.244461694960922])

In [7]:
kmeans(2,2,X)

purity, average distance, max distance to centroids and max distance between 2 feature vectors with L2 and k = 2


([0.9523809523809523, 0.9157894736842105],
 [0.7024990771498245, 0.5594128000997106],
 [2.034382770806892, 2.034382770806892],
 [3.1377420386244155])

In [8]:
kmeans(3,1,X)

purity, average distance, max distance to centroids and max distance between 2 feature vectors with L1 and k = 3


([0.918918918918919, 0.93048128342246, 0.9620253164556962],
 [3.875279432687562, 2.2266602794580534, 2.3146874907704693],
 [8.020198290493855, 8.020198290493855, 8.293228294563761],
 [14.244461694960922, 8.957694141301694])

In [ ]:
kmeans(3,2,X)

purity, average distance, max distance to centroids and max distance between 2 feature vectors with L2 and k = 3


In [ ]:
kmeans(4,1,X)

In [340]:
kmeans(4,2,X)

purity, average distance, max distance to centroids and max distance between 2 feature vectors with L2 and k = 4


([1.0, 0.9405099150141643, 1.0, 0.6951219512195121],
 [0.7140232013167087,
  0.4872262150394441,
  0.48761298789138896,
  0.5071185970475394],
 [1.6435738142241292,
  1.6435738142241292,
  1.6435738142241292,
  1.787389486980726],
 [2.242970349145292,
  1.8511590743415496,
  1.7968985335677565,
  2.299806124842556])

# Conclusions on Purity 

L2 distance seems to be performing better than L1 measure as the cumulative purity of all clusters is high for same value of K=2 and K=3. For K=4, there is not much to separate between L1 and L2 as the purity is 100% for 2 clusters and the difference in purity is cancelled out in the other two remaining clusters. The highest purity is achieved for K=3 using the L2 distance measure